In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

from uproot_io import Events, View
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

In [2]:
events = Events(r"C:\Users\andre\Desktop\Physics Project\bigdata\CheatedRecoFile_1.root")

In [3]:
training = np.load("training_events.npy")
testing = np.load("testing_events.npy")
truth = np.load("truth_data.npy")

In [11]:
training_truth = []
testing_truth = []
for event_number in training:
    training_truth.append(truth[event_number])
for event_number in testing:
    testing_truth.append(truth[event_number])

training_truth = np.array(training_truth)
testing_truth = np.array(testing_truth)

In [4]:
#for each event, find biggest particle, then calculate features of biggest particle

def biggest():
    biggest = []
    
    for n in training:
        nhits = 0
        big = []
        idx = np.where(events.event_number == n)[0]
        
        for i in idx:
            x = events.reco_hits_x_w[i]
            z = events.reco_hits_w[i]
            
            #could also try length
            if len(x) > nhits:
                nhits = len(x)
                big.append(i)
        
        biggest.append(big[-1])
    
    return np.array(biggest)

In [6]:
biggest = biggest()

In [8]:
print(len(biggest)), print(len(training))

6516
6516


(None, None)

In [65]:
def fit_line(x, y):
    A = np.vstack([x, np.ones_like(x)]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def calculate_residuals(x, y, m, c):
    #y_predicted = (m*x) + c
    perpdist = abs(((-m*x) + y - c) / np.sqrt(m**2 + 1))
    #residuals = np.abs(perpdist - y_predicted)
    return perpdist

def BDTdata():
    data = []
    eventlabel = []
    tslabel = []
    biggest = []
    party = []
    
    for n in training:
        nhits = 0
        big = []
        idx = np.where(events.event_number == n)[0]
        
        for i in idx:
            x = events.reco_hits_x_w[i]
            z = events.reco_hits_w[i]
            pdg = events.mc_pdg[i]
            purity = events.purity[i]
            complete = events.completeness[i]
            
            if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:
                if len(x) > nhits:
                    nhits = len(x)
                    big.append(i)
        
        if len(big) != 0:
            biggest.append(big[-1])

            x = events.reco_hits_x_w[big[-1]]
            z = events.reco_hits_w[big[-1]]
            pdg = events.mc_pdg[big[-1]]
            purity = events.purity[big[-1]]
            complete = events.completeness[big[-1]]

            if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:    
                party.append(pdg)
                if pdg == 22. or pdg == -11. or pdg == 11.:
                    tslabel.append(0) #showers
                else:
                    tslabel.append(1) #tracks
            
            if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:
                if truth[n] == 0:
                    eventlabel.append(0) #muon
                if truth[n] == 1:
                    eventlabel.append(1) #electron
                if truth[n] == 2:
                    eventlabel.append(2) #nc

#for n in training:
    #idx = np.where(events.event_number == n)[0]

    # line of best fit
    gradients = []
    intercepts = []

    xmax = -1000
    xmin = 1000
    zmax = -1000
    zmin = 1000
    for i in biggest:
        x0 = events.reco_hits_x_w[i]
        z0 = events.reco_hits_w[i]
        if len(x0) > 0:
            if max(x0) >= xmax:
                xmax = max(x0)
            if min(x0) <= xmin:
                xmin = min(x0)
        if len(z0) > 0:
            if max(z0) >= zmax:
                zmax = max(z0)
            if min(z0) <= zmin:
                zmin = min(z0)

    xlen = 0.1*(xmax - xmin)
    zlen = 0.1*(zmax - zmin)

    for i in biggest:

        # features
        entry = []

        # hits, pdg code, purity, completeness, adc values, vertex coordinates

        x = events.reco_hits_x_w[i]
        z = events.reco_hits_w[i]
        pdg = events.mc_pdg[i]
        purity = events.purity[i]
        complete = events.completeness[i]

        adc = events.reco_adcs_w[i]
        vx = events.reco_particle_vtx_x[i]
        vz = events.reco_particle_vtx_w[i]

        best_inliers = []
        best_model = None
        sample_size = 10
        max_iterations = 100

        for _ in range(max_iterations):
            sample_size = min(sample_size, len(x))
            sample_indices = np.random.choice(len(x), size=sample_size, replace=False)
            x_sample = x[sample_indices]
            z_sample = z[sample_indices]

            m, c = fit_line(x_sample, z_sample)
            residuals = calculate_residuals(x, z, m, c)
            inliers = np.where(residuals < (0.15*xlen))[0]

            if len(inliers) > len(best_inliers):
                best_inliers = inliers
                best_model = (m, c)

        if len(best_inliers) > 0:
            m, c = best_model

            gradients.append(m)
            intercepts.append(c)

            # changing the coordinates into longitudinal and tranverse from best line

            znew = z - c
            angle = -math.atan(m)
            l = x * np.cos(angle) - znew * np.sin(angle)
            t = x * np.sin(angle) + znew * np.cos(angle)

            # feature 1: proximal hits within 5% of transverse range of the hits.

            tdiff = np.abs(max(t)-min(t))
            margin = tdiff*0.05
            inside = [x for x in t if x <= margin and x >= -margin]
            entry.append(len(inside)/len(t))

            # feature 2: the standard deviation of the last 30% of a topology

            p1 = int(0.7 * len(t))
            X = t[p1:]
            Xvar = np.var(X)

            p2 = int(0.3 * len(t))
            Y = t[:p2]
            Yvar = np.var(Y)

            if  Yvar > Xvar:
                resultY = np.sqrt(Yvar)
                entry.append(resultY)           
            else:
                resultX = np.sqrt(Xvar)
                entry.append(resultX)

            # feature 3: the hits per length

            above = np.sum(t > 0)
            below = np.sum(t < 0)

            lmin, lmax = np.min(l), np.max(l)
            length = lmax - lmin

            if above == 0 or below == 0 or length == 0:
                entry.append(2)

            else:
                # If symmetric, this should be close to 1, otherwise > 1
                hits = (above + below) / length
                entry.append(hits)


            # feature 4: energy in the last 10% of the track

            vznew = vz - c

            vl = vx * np.cos(angle) - vznew * np.sin(angle)
            vt = vx * np.sin(angle) + vznew * np.cos(angle)

            if max(l) < vl:
                l *= -1

            combined = list(zip(l, t, adc))
            sort = sorted(combined, key = lambda coord: coord[0])

            lnew, tnew, adcnew = zip(*sort)

            percent = int(0.9 * len(adcnew))
            X = adcnew[percent:]
            Xsum = np.sum(X)

            total = np.sum(adcnew)
            result = Xsum / total

            entry.append(result)

            # feature 5: mean connection distance

            total_distance = 0.0
            for j in range(len(x) - 1):
                distance = np.sqrt((x[j+1] - x[j])**2 + (z[j+1] - z[j])**2)
                total_distance += distance
            meandist = total_distance/(len(x)-1)
            entry.append(meandist)
            data.append(entry)


        else:
            entry.append(0.1)  # proximal hits
            entry.append(1.5)  # standard deviation
            entry.append(2)    # hits per length
            entry.append(0.25) # energy
            entry.append(1)    # mean connection distance
            data.append(entry)

    return data, eventlabel, tslabel, party

In [66]:
data, eventlabel, tslabel, particle = BDTdata()

In [69]:
np.shape(tslabel)

(6403,)

In [47]:
x_train = data
y_train = tslabel

In [60]:
def fit_line(x, y):
    A = np.vstack([x, np.ones_like(x)]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def calculate_residuals(x, y, m, c):
    #y_predicted = (m*x) + c
    perpdist = abs(((-m*x) + y - c) / np.sqrt(m**2 + 1))
    #residuals = np.abs(perpdist - y_predicted)
    return perpdist

def BDTtestdata():
    data = []
    label = []
    biggest = []
    party = []
    
    for n in testing:
        nhits = 0
        big = []
        idx = np.where(events.event_number == n)[0]
        
        for i in idx:
            x = events.reco_hits_x_w[i]
            z = events.reco_hits_w[i]
            pdg = events.mc_pdg[i]
            purity = events.purity[i]
            complete = events.completeness[i]
            
            if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:
                if len(x) > nhits:
                    nhits = len(x)
                    big.append(i)
        
        if len(big) != 0:
            biggest.append(big[-1])

            x = events.reco_hits_x_w[big[-1]]
            z = events.reco_hits_w[big[-1]]
            pdg = events.mc_pdg[big[-1]]
            purity = events.purity[big[-1]]
            complete = events.completeness[big[-1]]

            if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:    
                party.append(pdg)
                if truth[n] == 0:
                    label.append(0)
                if truth[n] == 1:
                    label.append(1)
                if truth[n] == 2:
                    label.append(2)
            
#for n in training:
    #idx = np.where(events.event_number == n)[0]

    # line of best fit
    gradients = []
    intercepts = []

    xmax = -1000
    xmin = 1000
    zmax = -1000
    zmin = 1000
    for i in biggest:
        x0 = events.reco_hits_x_w[i]
        z0 = events.reco_hits_w[i]
        if len(x0) > 0:
            if max(x0) >= xmax:
                xmax = max(x0)
            if min(x0) <= xmin:
                xmin = min(x0)
        if len(z0) > 0:
            if max(z0) >= zmax:
                zmax = max(z0)
            if min(z0) <= zmin:
                zmin = min(z0)

    xlen = 0.1*(xmax - xmin)
    zlen = 0.1*(zmax - zmin)

    for i in biggest:

        # features
        entry = []

        # hits, pdg code, purity, completeness, adc values, vertex coordinates

        x = events.reco_hits_x_w[i]
        z = events.reco_hits_w[i]
        pdg = events.mc_pdg[i]
        purity = events.purity[i]
        complete = events.completeness[i]

        adc = events.reco_adcs_w[i]
        vx = events.reco_particle_vtx_x[i]
        vz = events.reco_particle_vtx_w[i]

        best_inliers = []
        best_model = None
        sample_size = 10
        max_iterations = 100

        for _ in range(max_iterations):
            sample_size = min(sample_size, len(x))
            sample_indices = np.random.choice(len(x), size=sample_size, replace=False)
            x_sample = x[sample_indices]
            z_sample = z[sample_indices]

            m, c = fit_line(x_sample, z_sample)
            residuals = calculate_residuals(x, z, m, c)
            inliers = np.where(residuals < (0.15*xlen))[0]

            if len(inliers) > len(best_inliers):
                best_inliers = inliers
                best_model = (m, c)

        if len(best_inliers) > 0:
            m, c = best_model

            gradients.append(m)
            intercepts.append(c)

            # changing the coordinates into longitudinal and tranverse from best line

            znew = z - c
            angle = -math.atan(m)
            l = x * np.cos(angle) - znew * np.sin(angle)
            t = x * np.sin(angle) + znew * np.cos(angle)

            # feature 1: proximal hits within 5% of transverse range of the hits.

            tdiff = np.abs(max(t)-min(t))
            margin = tdiff*0.05
            inside = [x for x in t if x <= margin and x >= -margin]
            entry.append(len(inside)/len(t))

            # feature 2: the standard deviation of the last 30% of a topology

            p1 = int(0.7 * len(t))
            X = t[p1:]
            Xvar = np.var(X)

            p2 = int(0.3 * len(t))
            Y = t[:p2]
            Yvar = np.var(Y)

            if  Yvar > Xvar:
                resultY = np.sqrt(Yvar)
                entry.append(resultY)           
            else:
                resultX = np.sqrt(Xvar)
                entry.append(resultX)

            # feature 3: the hits per length

            above = np.sum(t > 0)
            below = np.sum(t < 0)

            lmin, lmax = np.min(l), np.max(l)
            length = lmax - lmin

            if above == 0 or below == 0 or length == 0:
                entry.append(2)

            else:
                # If symmetric, this should be close to 1, otherwise > 1
                hits = (above + below) / length
                entry.append(hits)


            # feature 4: energy in the last 10% of the track

            vznew = vz - c

            vl = vx * np.cos(angle) - vznew * np.sin(angle)
            vt = vx * np.sin(angle) + vznew * np.cos(angle)

            if max(l) < vl:
                l *= -1

            combined = list(zip(l, t, adc))
            sort = sorted(combined, key = lambda coord: coord[0])

            lnew, tnew, adcnew = zip(*sort)

            percent = int(0.9 * len(adcnew))
            X = adcnew[percent:]
            Xsum = np.sum(X)

            total = np.sum(adcnew)
            result = Xsum / total

            entry.append(result)

            # feature 5: mean connection distance

            total_distance = 0.0
            for j in range(len(x) - 1):
                distance = np.sqrt((x[j+1] - x[j])**2 + (z[j+1] - z[j])**2)
                total_distance += distance
            meandist = total_distance/(len(x)-1)
            entry.append(meandist)
            data.append(entry)


        else:
            entry.append(0.1)  # proximal hits
            entry.append(1.5)  # standard deviation
            entry.append(2)    # hits per length
            entry.append(0.25) # energy
            entry.append(1)    # mean connection distance
            data.append(entry)

    return data, label, party

In [61]:
x_test, y_test, testpart = BDTtestdata()

In [62]:
len(x_test), len(y_test), len(testpart)

(2729, 2729, 2729)

In [63]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

trainpart = np.array(particle)
testpart = np.array(testpart)

keep1 = np.where(y_train != 2)
x_train_cut = x_train[keep1]
y_train_cut = y_train[keep1]

keep2 = np.where(y_test != 2)
x_test_cut = x_test[keep2]
y_test_cut = y_test[keep2]

In [64]:
np.save("x_train", x_train)
np.save("y_train", y_train)
np.save("x_test", x_test)
np.save("y_test", y_test)

np.save("x_train_cut", x_train_cut)
np.save("y_train_cut", y_train_cut)
np.save("x_test_cut", x_test_cut)
np.save("y_test_cut", y_test_cut)

np.save("trainpart", trainpart)
np.save("testpart", testpart)